In [0]:
corpus = 'assin-ptbr'
#assin-ptbr-rte, assin-ptpt-rte, tweetsent
model = 'neuralmind/bert-base-portuguese-cased' 
#neuralmind/bert-base-portuguese-cased, bert-base-multilingual-cased, roberta-large

filepath = f'output/{model}/{corpus}-rte/predictions.json'
filepaths = {'rte': filepath}

output_dir = f'report/{model}/'

download_folder = 'data/dataset'

Get dataset and requirement

In [12]:
!wget http://nilc.icmc.usp.br/assin/assin.tar.gz
!tar -xzf assin.tar.gz
%rm -rf assin.tar.gz

!pip install xmltodict

%mkdir $download_folder
%mv *.xml $download_folder

--2020-04-20 21:54:49--  http://nilc.icmc.usp.br/assin/assin.tar.gz
Resolving nilc.icmc.usp.br (nilc.icmc.usp.br)... 143.107.183.225
Connecting to nilc.icmc.usp.br (nilc.icmc.usp.br)|143.107.183.225|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 749735 (732K) [application/x-gzip]
Saving to: ‘assin.tar.gz’

assin.tar.gz        100%[===================>] 732.16K  1.06MB/s    in 0.7s    

2020-04-20 21:54:50 (1.06 MB/s) - ‘assin.tar.gz’ saved [749735/749735]



Import xml files

In [0]:
import xmltodict
import json

tasks = ['rte']
scores = dict()

for task in tasks:
    filepath = filepaths[task]
    with open(filepath) as f:
        scores[task] = json.load(f)

    goldfile = 'assin2-blind-test.xml' if corpus == 'assin2' else f'{corpus}-test.xml'
    with open(f'{download_folder}/{goldfile}') as f:
        xml = xmltodict.parse(f.read())

Insert scores

In [0]:
for idx, item in enumerate(xml['entailment-corpus']['pair']):
    entailment_labels = ['Entailment','None','Paraphrase']
    entailment = entailment_labels[scores['rte'][idx]]

    xml['entailment-corpus']['pair'][idx]['@entailment'] = entailment

Save XML

In [0]:
result = xmltodict.unparse(xml, pretty = True)
with open(f'{output_dir}/{corpus}-test.xml', 'w') as f:
    f.write(result)